In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [34]:

#data = pd.read_excel('data_inspection.xlsx')
data = pd.read_excel('sample.xlsx', sheet_name=1)


In [35]:
cols = ~data.where(data.isin(["REMARKS"])).isnull().all()
#target_cols = cols[:np.argmax(cols)+1].index


In [38]:
cols.idxmax()

'Unnamed: 20'

In [39]:
def check_string(df, string):
    return (~df.where(data.isin([string])).isnull().all()).any()

def find_ids(df, string):
    # check string if available
    if not check_string(df, string):
        print(f"The string of '{string}' is not found.")
        return None
    # find cols contain string
    cols = ~df.where(df.isin([string])).isnull().all()
    target_col = cols.index[cols].values
    
    # extract df that columns contain string
    target_df = df[target_col]
    bools = target_df.isin([string]).values.flatten()
    
    #extract indexs contains string
    ids = target_df[bools].index.values
    
    return ids

def get_start_end_ids(df):
    start_ids = find_ids(df, "Desc.")

    if check_string(df, "Note :"):
        end_ids = find_ids(data, "Note:")

    else:
        if check_string(df, "Inspected By :"):
            end_ids = find_ids(df, "Inspected By :") 
        elif check_string(df, "Inspected by :"):
            end_ids = find_ids(df, "Inspected by :") - 4
            
    return start_ids, end_ids

def get_data(df, start_ids, end_ids):
    # get only df up to columns "REMARKS"
    cols = ~df.where(df.isin(["REMARKS"])).isnull().all()
    target_cols = cols[:np.argmax(cols.values)+1].index
    df = df[target_cols]
    df = df.replace(0, np.nan)
    
    for i, (start, end) in enumerate(list(zip(start_ids, end_ids))):
        subset_df = df.loc[start+2:end-1]
        if i == 0:
            df_ = subset_df
        else:
            df_ = pd.concat([df_, subset_df], axis=0)
            
    df_ = df_.dropna(how="all", axis=0).dropna(how="all", axis=1)
    

    
    # rename columns
    col_names = ['section', 'deg'] + [f'CML{i+1}' for i in range(len(df_.columns)-5)] + ['min_thick', "min_thick_circum", "remarks"]
    df_.columns = col_names
    
    # coerce numeric convert
    for col in [name for name in df_.columns if "CML" in name ]:
        df_[col] = pd.to_numeric(df_[col], errors="coerce")
    
    # remove "-" value
    df_ = df_.replace("-", np.nan)
    df_ = df_.dropna(how="all", axis=0)
    df_ = df_.reset_index(drop=True)
    return df_

def preprocess_section(df):
    section_ls = df.dropna(subset=["section"]).section.tolist()
    index_ls = df.section[df.section.notnull()].index

    df_ = pd.DataFrame(columns = df.columns)
    for i in range(len(index_ls)):

        start = index_ls[i]

        if i < len(index_ls) -1:
            end = index_ls[i+1]
            temp_df = df.loc[start:end-1, :]


        else:
            temp_df = df.loc[start:, :]

        temp_df['section'] = section_ls[i]
        df_ = pd.concat([df_, temp_df], axis=0)

    df_.loc[:, "section_group"] = df_['section']
    df_.loc[df_.section.str.contains("SHELL|SHEEL"), "section_group"] = "SHELL"
    df_.loc[df_.section.str.contains("NOZZLE"), "section_group"] = "NOZZLE"
    return df_

def get_nozzle_df(df):
    nozzle = df[(df.section.str.contains("NOZZLE")) & (df.deg.str.contains("N|MH"))]
    nozzle['NPS'] = nozzle.remarks.str.extract('(\d+)')
    return nozzle

def get_pv_df(df):
    pv = df[~df.section.str.contains("NOZZLE")]
    return pv

def nozzle_summary(df):
    nozzle_nom = df.groupby('NPS').max().max(axis=1).to_frame()
    nozzle_nom.columns = ['t_nom']
    nozzle_ = df.merge(nozzle_nom, how="left", on=['NPS'])
    nozzle_t_nom = nozzle_['t_nom'].tolist()
    nozzle_names = nozzle_['deg'].tolist()
    nozzle_size = nozzle_['NPS'].astype(int).tolist()
    nozzle_thick = nozzle_.min(axis=1).tolist()
    summary = dict()
    summary['nozzle_names'] = nozzle_names
    summary['nozzle_size'] = nozzle_size
    summary['nozzles_thick_nom'] = nozzle_t_nom
    summary['nozzles_thick'] = nozzle_thick
    return summary

def print_summary_nozzle(df):
    summary_dict = nozzle_summary(df)
    
    print(f"""
Nozzles Summary (size in inch, thick in mm)
===========================================""")
    for k, v in summary_dict.items():
        print(f"{k} = {v}")
        
def print_summary_pv(df):
    summary = df.groupby('section_group').min().min(axis=1)
    noms = df.groupby('section_group').max().max(axis=1)
    print(f"""
Summary Min Thickness (mm)
===========================
{summary}

Possible Nominal Thickness (mm)
===============================
{noms}""")
    

def print_all(pv_df, nozzle_df):
    print_summary_pv(pv_df)
    print_summary_nozzle(nozzle_df)


        
start_ids, end_ids = get_start_end_ids(data)
data1 = get_data(data, start_ids, end_ids)
data2 = preprocess_section(data1)
nozzle = get_nozzle_df(data2)
pv = get_pv_df(data2)

print_all(pv, nozzle)



Summary Min Thickness (mm)
section_group
HEAD 1A    11.51
HEAD 1B    11.82
HEAD 2A    11.80
HEAD 2B    11.39
SHELL       9.32
dtype: float64

Possible Nominal Thickness (mm)
section_group
HEAD 1A    12.29
HEAD 1B    12.27
HEAD 2A    12.28
HEAD 2B    12.15
SHELL       9.62
dtype: float64

Nozzles Summary (size in inch, thick in mm)
nozzle_names = ['N1', 'N2', 'MH']
nozzle_size = [4, 4, 16]
nozzles_thick_nom = [6.03, 6.03, 9.46]
nozzles_thick = [5.64, 5.59, 9.37]


In [40]:
class Inspection:
    def __init__(self, path, sheet_name=0):
        self.data = pd.read_excel(path, sheet_name= sheet_name)
        self.get_start_end_ids()
        self.data1 = self.get_data(self.data, self.start_ids, self.end_ids)
        self.data2 = self.preprocess_section(self.data1)
        
        # process nozzle 
        self.nozzle = self.get_nozzle_df(self.data2)
        self.nozzle = self.nozzle.dropna(how="all", axis=1)
        self.nozzle = self.nozzle.reset_index(drop=True)
        
        # proces pv dataframe
        self.pv = self.get_pv_df(self.data2)
        pv_null = self.pv.isnull().sum(axis=1)
        self.pv = self.pv.drop(index=pv_null[pv_null > 10].index)
        self.pv = self.pv.reset_index(drop=True)
        
    def check_string(self, df, string):
        return (~df.where(df.isin([string])).isnull().all()).any()

    def find_ids(self, df, string):
        # check string if available
        if not self.check_string(df, string):
            print(f"The string of '{string}' is not found.")
            return None
        # find cols contain string
        cols = ~df.where(df.isin([string])).isnull().all()
        target_col = cols.index[cols].values

        # extract df that columns contain string
        target_df = df[target_col]
        bools = target_df.isin([string]).values.flatten()

        #extract indexs contains string
        ids = target_df[bools].index.values

        return ids

    def get_start_end_ids(self):
        self.start_ids = self.find_ids(self.data, "Desc.")

        if self.check_string(self.data, "Note :"):
            self.end_ids = self.find_ids(self.data, "Note:")
            print(self.end_ids)

        else:
            if self.check_string(self.data, "Inspected By :"):
                self.end_ids = self.find_ids(self.data, "Inspected By :") - 4
            elif self.check_string(self.data, "Inspected by :"):
                self.end_ids = self.find_ids(self.data, "Inspected by :") - 4



    def get_data(self, df, start_ids, end_ids):
        # get only df up to columns "REMARKS"
        cols = ~df.where(df.isin(["REMARKS"])).isnull().all()
        target_cols = cols[:np.argmax(cols.values)+1].index
        df = df[target_cols]

        for i, (start, end) in enumerate(list(zip(start_ids, end_ids))):
            subset_df = df.loc[start+2:end-1]
            if i == 0:
                df_ = subset_df
            else:
                df_ = pd.concat([df_, subset_df], axis=0)

        df_ = df_.dropna(how="all", axis=0).dropna(how="all", axis=1)



        # rename columns
        col_names = ['section', 'deg'] + [f'CML{i+1}' for i in range(len(df_.columns)-5)] + ['min_thick', "min_thick_circum", "remarks"]
        df_.columns = col_names

        # coerce numeric convert
        for col in [name for name in df_.columns if "CML" in name ]:
            df_[col] = pd.to_numeric(df_[col], errors="coerce")

        # remove "-" value
        df_ = df_.replace("-", np.nan)
        df_ = df_.dropna(how="all", axis=0)
        df_ = df_.reset_index(drop=True)
        return df_

    def preprocess_section(self, df):
        section_ls = df.dropna(subset=["section"]).section.tolist()
        index_ls = df.section[df.section.notnull()].index

        df_ = pd.DataFrame(columns = df.columns)
        for i in range(len(index_ls)):

            start = index_ls[i]

            if i < len(index_ls) -1:
                end = index_ls[i+1]
                temp_df = df.loc[start:end-1, :]


            else:
                temp_df = df.loc[start:, :]

            temp_df['section'] = section_ls[i]
            df_ = pd.concat([df_, temp_df], axis=0)

        df_.loc[:, "section_group"] = df_['section']
        df_.loc[df_.section.str.contains("SHELL|SHEEL"), "section_group"] = "SHELL"
        df_.loc[df_.section.str.contains("NOZZLE"), "section_group"] = "NOZZLE"
        return df_

    def get_nozzle_df(self, df):
        nozzle = df[(df.section.str.contains("NOZZLE")) & (df.deg.str.contains("N|MH"))]
        nozzle['NPS'] = nozzle.remarks.str.extract('(\d+)')
        return nozzle

    def get_pv_df(self, df):
        pv = df[~df.section.str.contains("NOZZLE")]
        return pv

    def nozzle_summary(self, df):
        nozzle_nom = df.groupby('NPS').max().max(axis=1).to_frame()
        nozzle_nom.columns = ['t_nom']
        nozzle_ = df.merge(nozzle_nom, how="left", on=['NPS'])
        nozzle_t_nom = nozzle_['t_nom'].tolist()
        nozzle_names = nozzle_['deg'].tolist()
        nozzle_size = nozzle_['NPS'].astype(int).tolist()
        nozzle_thick = nozzle_.min(axis=1).tolist()
        summary = dict()
        summary['nozzle_names'] = nozzle_names
        summary['nozzle_size'] = nozzle_size
        summary['nozzles_thick_nom'] = nozzle_t_nom
        summary['nozzles_thick'] = nozzle_thick
        return summary

    def print_summary_nozzle(self, df):
        summary_dict = self.nozzle_summary(df)

        print(f"""
Nozzles Summary (size in inch, thick in mm)
===========================================""")
        for k, v in summary_dict.items():
            print(f"{k} = {v}")

    def print_summary_pv(self, df):
        summary = df.groupby('section_group').min().min(axis=1)
        noms = df.groupby('section_group').max().max(axis=1)
        print(f"""
Summary Min Thickness (mm)
===========================
    {summary}

Possible Nominal Thickness (mm)
===============================
    {noms}""")


    def print_all(self):
        self.print_summary_pv(self.pv)
        self.print_summary_nozzle(self.nozzle)



test = Inspection("sample.xlsx", sheet_name=1)
test.print_all()


Summary Min Thickness (mm)
    section_group
HEAD 1A    11.51
HEAD 1B    11.82
HEAD 2A    11.80
HEAD 2B    11.39
SHELL       9.32
dtype: float64

Possible Nominal Thickness (mm)
    section_group
HEAD 1A    12.29
HEAD 1B    12.27
HEAD 2A    12.28
HEAD 2B    12.15
SHELL       9.62
dtype: float64

Nozzles Summary (size in inch, thick in mm)
nozzle_names = ['N1', 'N2', 'MH']
nozzle_size = [4, 4, 16]
nozzles_thick_nom = [6.03, 6.03, 9.46]
nozzles_thick = [5.64, 5.59, 9.37]


,section,deg,CML1,CML2,CML3,CML4,CML5,CML6,CML7,CML8,...,CML11,CML12,CML13,CML14,CML15,CML16,min_thick,min_thick_circum,remarks,section_group
0,TOP HEAD A,0º,10.71,10.66,10.68,10.65,10.65,11.49,11.57,11.50,...,11.36,10.59,10.55,10.58,11.48,10.63,10.55,10.41,NaN,TOP HEAD A
1,TOP HEAD A,90º,10.63,11.02,11.10,10.65,10.73,10.64,10.68,10.72,...,10.68,10.64,11.16,11.03,10.65,10.73,10.63,NaN,NaN,TOP HEAD A
2,TOP HEAD A,180º,11.36,11.20,11.40,11.17,11.37,11.02,11.49,11.03,...,11.36,11.45,11.32,11.40,11.32,10.58,10.53,NaN,NaN,TOP HEAD A
3,TOP HEAD A,270º,10.96,10.41,11.02,11.08,11.21,11.28,10.46,11.21,...,11.09,11.01,11.05,11.06,11.08,10.88,10.41,NaN,NaN,TOP HEAD A
4,TOP HEAD B,0º,11.07,11.01,11.07,11.03,11.01,11.08,11.03,11.09,...,11.10,11.14,11.05,11.13,11.14,11.25,11.01,10.38,NaN,TOP HEAD B
5,TOP HEAD B,90º,10.98,10.92,10.94,11.00,10.96,11.09,11.02,10.93,...,11.01,11.09,10.97,11.03,10.85,10.94,10.85,NaN,NaN,TOP HEAD B
6,TOP HEAD B,180º,11.31,11.25,11.30,11.34,10.93,10.95,11.18,11.19,...,11.28,10.46,10.45,11.10,10.63,11.09,10.45,NaN,NaN,TOP HEAD B
7,TOP HEAD B,270º,10.85,11.07,11.06,10.94,10.97,10.38,11.05,11.01,...,10.96,11.11,11.10,11.13,11.17,11.16,10.38,NaN,NaN,TOP HEAD B
8,SHELL 1A,0º,9.64,9.60,9.66,9.62,9.63,9.64,9.60,9.65,...,9.62,9.64,9.63,9.59,9.64,9.60,9.59,7.92,NaN,SHELL
9,SHELL 1A,90º,9.59,9.57,9.60,9.57,9.57,9.58,9.56,9.54,...,7.92,9.66,9.63,9.62,9.64,9.63,7.92,NaN,NaN,SHELL


In [332]:
section_ls = data1.dropna(subset=["section"]).section.tolist()
index_ls = data1.section[data1.section.notnull()].index

data2 = pd.DataFrame(columns = data1.columns)
for i in range(len(index_ls)):
    
    start = index_ls[i]
    
    if i < len(index_ls) -1:
        end = index_ls[i+1]
        temp_df = data1.loc[start:end-1, :]
        
        
    else:
        temp_df = data1.loc[start:, :]
    
    temp_df['section'] = section_ls[i]
    data2 = pd.concat([data2, temp_df], axis=0)

data2.loc[:, "section_group"] = data2['section']
data2.loc[data2.section.str.contains("SHELL"), "section_group"] = "SHELL"
data2.loc[data2.section.str.contains("NOZZLE"), "section_group"] = "NOZZLE"
data2

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,section,deg,CML1,CML2,CML3,CML4,CML5,CML6,CML7,CML8,...,CML11,CML12,CML13,CML14,CML15,CML16,min_thick,min_thick_circum,remarks,section_group
0,TOP HEAD A,0º,10.71,10.66,10.68,10.65,10.65,11.49,11.57,11.50,...,11.36,10.59,10.55,10.58,11.48,10.63,10.55,10.41,NaN,TOP HEAD A
1,TOP HEAD A,90º,10.63,11.02,11.10,10.65,10.73,10.64,10.68,10.72,...,10.68,10.64,11.16,11.03,10.65,10.73,10.63,NaN,NaN,TOP HEAD A
2,TOP HEAD A,180º,11.36,11.20,11.40,11.17,11.37,11.02,11.49,11.03,...,11.36,11.45,11.32,11.40,11.32,10.58,10.53,NaN,NaN,TOP HEAD A
3,TOP HEAD A,270º,10.96,10.41,11.02,11.08,11.21,11.28,10.46,11.21,...,11.09,11.01,11.05,11.06,11.08,10.88,10.41,NaN,NaN,TOP HEAD A
4,TOP HEAD B,0º,11.07,11.01,11.07,11.03,11.01,11.08,11.03,11.09,...,11.10,11.14,11.05,11.13,11.14,11.25,11.01,10.38,NaN,TOP HEAD B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,"MAIN HOLE, REINFORCEMENT and NOZZLE",PAD,23.30,NaN,NaN,NaN,23.32,NaN,NaN,NaN,...,NaN,NaN,23.32,NaN,NaN,NaN,23.26,NaN,NaN,NOZZLE
76,"MAIN HOLE, REINFORCEMENT and NOZZLE",MH2,10.60,NaN,NaN,NaN,9.95,NaN,NaN,NaN,...,NaN,NaN,9.92,NaN,NaN,NaN,9.92,NaN,"Ø 24""",NOZZLE
77,"MAIN HOLE, REINFORCEMENT and NOZZLE",PAD,23.12,NaN,NaN,NaN,23.19,NaN,NaN,NaN,...,NaN,NaN,23.18,NaN,NaN,NaN,23.12,NaN,NaN,NOZZLE
78,"MAIN HOLE, REINFORCEMENT and NOZZLE",MH3,9.85,NaN,NaN,NaN,9.93,NaN,NaN,NaN,...,NaN,NaN,9.73,NaN,NaN,NaN,9.73,NaN,"Ø 26""",NOZZLE


In [333]:
nozzle = data2[(data2.section.str.contains("NOZZLE")) & (data2.deg.str.contains("N|MH"))]
nozzle['NPS'] = nozzle.remarks.str.extract('(\d+)')
pv = data2[~data2.section.str.contains("NOZZLE")]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [334]:
pv.groupby('section_group').min().min(axis=1)

section_group
BOTTOM HEAD A    11.07
BOTTOM HEAD B    11.08
SHELL             7.35
TOP    HEAD A    10.41
TOP    HEAD B    10.38
dtype: float64

In [335]:
pv.groupby('section_group').max().max(axis=1)

section_group
BOTTOM HEAD A    11.30
BOTTOM HEAD B    11.29
SHELL            10.54
TOP    HEAD A    11.57
TOP    HEAD B    11.34
dtype: float64

In [339]:
nozzle_nom = nozzle.groupby('NPS').max().max(axis=1).to_frame()
nozzle_nom.columns = ['t_nom']
nozzle_ = nozzle.merge(nozzle_nom, how="left", on=['NPS'])
nozzle_t_nom = nozzle_['t_nom'].tolist()
nozzle_names = nozzle_['deg'].tolist()
nozzle_size = nozzle_['NPS'].tolist()
nozzle_thick = nozzle_.min(axis=1).tolist()
nozzle_names, nozzle_size, nozzle_t_nom, nozzle_thick

(['N1', 'N2', 'N3', 'N4', 'N12', 'N13', 'MH1', 'MH2', 'MH3'],
 ['3', '4', '3', '4', '4', '3', '26', '24', '26'],
 [8.78, 10.02, 8.78, 10.02, 10.02, 8.78, 9.93, 10.6, 9.93],
 [7.02, 9.9, 7.63, 8.85, 8.61, 8.74, 8.13, 9.92, 9.73])

0    7.02
1    9.90
2    7.63
3    8.85
4    8.61
5    8.74
6    8.13
7    9.92
8    9.73
dtype: float64

In [291]:
nozzle

,section,deg,CML1,CML2,CML3,CML4,CML5,CML6,CML7,CML8,...,CML11,CML12,CML13,CML14,CML15,CML16,min_thick,min_thick_circum,remarks,section_group
61,"MAIN HOLE, REINFORCEMENT and NOZZLE",N1,7.02,NaN,NaN,NaN,8.41,NaN,NaN,NaN,...,NaN,NaN,7.77,NaN,NaN,NaN,7.02,NaN,3,NOZZLE
63,"MAIN HOLE, REINFORCEMENT and NOZZLE",N2,9.95,NaN,NaN,NaN,9.93,NaN,NaN,NaN,...,NaN,NaN,9.90,NaN,NaN,NaN,9.90,NaN,4,NOZZLE
65,"MAIN HOLE, REINFORCEMENT and NOZZLE",N3,7.63,NaN,NaN,NaN,7.80,NaN,NaN,NaN,...,NaN,NaN,8.06,NaN,NaN,NaN,7.63,NaN,3,NOZZLE
67,"MAIN HOLE, REINFORCEMENT and NOZZLE",N4,8.90,NaN,NaN,NaN,8.99,NaN,NaN,NaN,...,NaN,NaN,8.85,NaN,NaN,NaN,8.85,NaN,4,NOZZLE
69,"MAIN HOLE, REINFORCEMENT and NOZZLE",N12,9.13,NaN,NaN,NaN,9.62,NaN,NaN,NaN,...,NaN,NaN,9.52,NaN,NaN,NaN,8.61,NaN,4,NOZZLE
72,"MAIN HOLE, REINFORCEMENT and NOZZLE",N13,8.76,NaN,NaN,NaN,8.74,NaN,NaN,NaN,...,NaN,NaN,8.78,NaN,NaN,NaN,8.74,NaN,"Ø 3""",NOZZLE
74,"MAIN HOLE, REINFORCEMENT and NOZZLE",MH1,8.13,NaN,NaN,NaN,9.14,NaN,NaN,NaN,...,NaN,NaN,9.73,NaN,NaN,NaN,8.13,NaN,"Ø 26""",NOZZLE
76,"MAIN HOLE, REINFORCEMENT and NOZZLE",MH2,10.60,NaN,NaN,NaN,9.95,NaN,NaN,NaN,...,NaN,NaN,9.92,NaN,NaN,NaN,9.92,NaN,"Ø 24""",NOZZLE
78,"MAIN HOLE, REINFORCEMENT and NOZZLE",MH3,9.85,NaN,NaN,NaN,9.93,NaN,NaN,NaN,...,NaN,NaN,9.73,NaN,NaN,NaN,9.73,NaN,"Ø 26""",NOZZLE
